In [33]:
from mysql.connector import Error, MySQLConnection
from configparser import ConfigParser 

In [32]:
def connection_config(filename='config.ini', section='mysql'):
    """ читаем файл конфигурации и возвращаем указанные в нем параметры в качестве словаря
    """    
    parser = ConfigParser()
    parser.read(filename)
    db = {}
    if parser.has_section(section):
        items = parser.items(section)
        for item in items:
            db[item[0]] = item[1]
    else:
        raise Exception(f'{section} not found in the {filename} file')
 
    return db

In [42]:
def connect_db():
    """установка соединения с БД"""

    db_config = connection_config()

    try:
        print('Connecting to MySQL database...')
        conn = MySQLConnection(**db_config)
        with conn:
            if conn.is_connected():
                print('connection established.')
            else:
                print('connection failed.')
    except Error as error:
        print(error)
    finally:       
        print('Connection closed.')

In [43]:
connect_db() #проверка

Connecting to MySQL database...
connection established.
Connection closed.


### Create

In [67]:
def create_smth(query):
    try:
        dbconfig = connection_config()
        conn = MySQLConnection(**dbconfig)
        with conn:
            cursor = conn.cursor()
            cursor.execute(query)
    except Error as e:
        print(e)
        
    
def insert_data(query, inputs):
    try:
        db_config = connection_config()
        conn = MySQLConnection(**db_config)
        with conn:
            cursor = conn.cursor()
            if len(inputs)<1:
                cursor.execute(query, inputs)
            else:
                cursor.executemany(query, inputs)            
            conn.commit()
    except Error as error:
        print(error)


    
query = """CREATE TABLE IF NOT EXISTS test_connector 
(
id INT PRIMARY KEY AUTO_INCREMENT,
test_num VARCHAR(45),
result BOOLEAN NOT NULL DEFAULT 1
);"""

query2 = """INSERT INTO test_connector(test_num,result) VALUES (%s,%s);"""
inputs = [(1,0),(2,0),(3,1)]

if __name__ == '__main__':
    main()

### Read

In [68]:
def iter_row(cursor, size=10):
    while True:
        rows = cursor.fetchmany(size)
        if not rows:
            break
        for row in rows:
            yield row

def read_db(query):
    try:
        dbconfig = connection_config()
        conn = MySQLConnection(**dbconfig)
        with conn:
            cursor = conn.cursor()
            cursor.execute(query)
            for row in iter_row(cursor, 10):
                print(row)

    except Error as e:
        print(e)

query3 = """
SELECT * from test_connector
"""

read_db(query3)

(9, '1', 0)
(10, '1', 0)
(11, '2', 0)
(12, '3', 1)
(13, '1', 0)
(14, '2', 0)
(15, '3', 1)
(16, '1', 0)
(17, '2', 0)
(18, '3', 1)


### Update

In [82]:
def upd_alt_del_db(query):    
    db_config = connection_config()
    try:
        conn = MySQLConnection(**db_config)
        with conn:
            cursor = conn.cursor()
            cursor.execute(query)
            conn.commit()

    except Error as error:
        print(error)

query4 = """ALTER TABLE test_connector
DROP COLUMN test_num;
"""
query5 = """ALTER TABLE test_connector
ADD COLUMN test_res VARCHAR(45) NOT NULL;
"""

query6 = """UPDATE test_connector
SET test_res = IF (result = 1, 'ok', 'not ok');
"""

### Delete

In [83]:
#удалять данные можем с помощью тех же действий, что и update 
query7 = """DELETE FROM test_connector
where test_res = 'not ok'
"""
upd_alt_del_db(query7)

In [ ]:
def main():
    create_smth(query)
    insert_data(query2, inputs)
    read_db(query3)
    upd_alt_del_db(query4)
    upd_alt_del_db(query5)
    upd_alt_del_db(query6)
    upd_alt_del_db(query7)